# Lesson: Safely Removing Data in MongoDB

### Lesson Introduction and Overview

Welcome, future data experts! Today's session explores MongoDB's data deletion. Think of deletion as a space cleanup — removing outdated 'stars' or documents from our 'cosmos' database. Our aim is to learn and master data removal in MongoDB, a crucial step in database management. Let's start our voyage!

### Understanding Deletion in MongoDB

Imagine a scenario where a satellite finishes its mission. In our database, its data is no longer relevant. To keep our data up-to-date, we must 'decommission' this satellite's data from our collection. Here in MongoDB, we refer to this removal operation as 'deletion'. It might seem simple, but wielded correctly, deletion becomes a powerful tool in database management.

### Precautions in Removing Data

Necessary precautions should be taken before deletion, beginning with a fresh data backup. Deletion can be an irreversible operation, and a backup ensures there's a recovery option in the event of unintended mass deletion.

### Deleting Documents Using `deleteOne()` and `deleteMany()`

We use Mongoose's deleteOne() and deleteMany() in MongoDB for deletion. The deleteOne() method deletes the first document matching the condition. Let's pretend we're decommissioning a satellite named 'OldStar' from our Satellite collection:

In [ ]:
const mongoose = require('mongoose'); // Importing Mongoose
const Satellite = mongoose.model('Satellite'); // Getting the Satellite collection

// Deleting 'OldStar' satellite document
Satellite.deleteOne({ name: 'OldStar' })
  .then(() => console.log('OldStar has been decommissioned.'))
  .catch((err) => console.error(err));

// The deleteMany() method deletes all the documents matching a given condition. Suppose we're decommissioning all inactive satellites:

// Deleting all inactive satellites
Satellite.deleteMany({ isActive: false })
  .then(() => console.log('All inactive satellites have been decommissioned.'))
  .catch((err) => console.error(err));

### Checking Deletion Accuracy with Delete Count

MongoDB's deletedCount property of the deletion response lets us verify the number of documents deleted. This aids in ensuring the accuracy of our operation:

In [ ]:
// Deletes all inactive satellites and checks deletion count
Satellite.deleteMany({ isActive: false })
  .then((result) => console.log("Documents deleted: " + result.deletedCount))
  .catch((err) => console.error(err)); 

// Tip: Always verify your operations for accuracy.

### Dropping Collections in MongoDB

In specific scenarios, you might need to delete an entire MongoDB collection. In such cases, we can use the drop() function:

In [ ]:
// Dropping the Satellite collection
Satellite.collection.drop()
  .then(() => console.log('Satellite collection successfully dropped!'))
  .catch((err) => console.error(err));

// Note: Dropping a collection is irreversible. Use with caution!

Remember, dropping a collection deletes the entire collection itself and can't be undone. For safety, always maintain a backup before executing the drop() operation.

### Lesson Summary & Upcoming Practice

Congratulations on completing this session on MongoDB's data deletion operations! We traversed the essence of deletion in MongoDB, learned about taking precautions before undertaking deletion, dove into MongoDB's deleteOne() and deleteMany() methods, and wrapped up by learning to verify our deletion operations.

Remember that deletion keeps our cosmic MongoDB database tidy by cleaning out old 'stars', thereby making our database explorations smoother. However, the power of deletion must be used wisely, keeping in mind potential repercussions.

The next crucial part is applying what you learned today in real-world scenarios through practice. Buckle up for practical exercises coming next, and let's continue on our informative MongoDB journey!

### Exercises:

In the code provided, we simulate the decommissioning of giant planets from our planets collection. The backend server is set up to handle a DELETE request, which, when invoked, removes any planet that has a mass greater than 10. Click Run to execute the code and observe how giant planets are cleared from our galactic database!

In [ ]:
// index.js (Express server)
const express = require('express');
const mongoose = require('mongoose');
const cors = require('cors');
const initialDBSetup = require('./initialDBSetup');

const app = express();
const port = 5000;

// Connect to MongoDB
mongoose.connect('mongodb://127.0.0.1/myDB', { useNewUrlParser: true, useUnifiedTopology: true }).then(async () => {
  console.log('connected');
  // initialize DB
  await initialDBSetup();
});

const Planet = mongoose.model('Planet', new mongoose.Schema({
  name: String,
  size: String,
  mass: Number,
  distanceFromSun: String
}));

app.use(cors());
app.use(express.json());

app.delete('/api/delete-giant-planets', (req, res) => {
  Planet.deleteMany({ mass: { $gt: 10 } })
    .then(result => res.send({ deletedCount: result.deletedCount }))
    .catch(err => {
      console.error(err);
      res.status(500).send(err);
    });
});

app.listen(port, () => {
  console.log(`Server listening at http://localhost:${port}`);
});

In [ ]:
// App.js (React frontend)
import { useState, useEffect } from 'react';
import axios from 'axios';

function App() {
  const [deletedStatus, setDeletedStatus] = useState("");
  
  const handleDelete = () => {
    axios
      .delete("/api/delete-giant-planets")
      .then(response => setDeletedStatus(`Deleted ${response.data.deletedCount} heavy planets!`))
      .catch(error => console.error("Error deleting:", error));
  };
  
  return (
    <div className="App">
      <button onClick={handleDelete}>Decommission Heavy Planets</button>
      <p>{deletedStatus}</p>
    </div>
  );
}

export default App;

Space Voyager, you've learned how to decommission a satellite by its activeness. Can you change the code to decommission the specified satellite? Modify the condition in the delete operation to utilize the satellite's name instead of the isActive field.

In [ ]:
// index.js (Express server)
const express = require('express');
const mongoose = require('mongoose');
const SatelliteSchema = new mongoose.Schema({ name: String, isActive: Boolean }, { versionKey: false });
const Satellite = mongoose.model('Satellite', SatelliteSchema);

const app = express();

mongoose.connect('mongodb://127.0.0.1/myDB', { useNewUrlParser: true, useUnifiedTopology: true }).then( async () => {
  console.log('Connected to MongoDB');
  
  Satellite.deleteMany({}).then(async () => {
    // Insert some initial satellites on first connection
    await Satellite.insertMany([
      { name: 'Hubble',  isActive: false },
      { name: 'Voyager1',isActive: false },
      { name: 'Juno', isActive: true }
    ]);
  });
});

app.use(express.json());

app.get('/api/satellites', async (req, res) => {
  try {
    const satellites = await Satellite.find();
    res.status(200).json(satellites);
  } catch (error) {
    res.status(500).json({ message: 'Failed to retrieve satellites.' });
  }
});

app.delete('/api/delete-satellites/:name', async (req, res) => {
  try {
    const result = await Satellite.deleteMany({ name : req.params.name }); // updated from isActive: false to name : req.params.name
    if (result.deletedCount > 0) {
      res.status(200).json({ message: `Successfully decommissioned satellite: ${req.params.name}` });
    } else {
      res.status(404).json({ message: 'Satellite not found.' });
    }
  } catch (error) {
    res.status(500).json({ message: 'Failed to decommission satellite.' });
  }
});

app.listen(5000, () => console.log('Server running on port 5000'));

In [ ]:
// App.js

import React, { useState, useEffect } from 'react';
import axios from 'axios';

function App() {
  const [satellites, setSatellites] = useState([]);

  useEffect(() => {
    fetchSatellites();
  }, []);

  const fetchSatellites = () => {
    axios.get('/api/satellites')
      .then(response => setSatellites(response.data))
      .catch(error => console.error('Error fetching satellites:', error.message));
  };

  const handleDecommission = (satelliteName) => {
    axios.delete(`/api/delete-satellites/${satelliteName}`)
      .then(() => {
        fetchSatellites(); // Refetch the satellites to get the updated list
      })
      .catch(error => console.error('Error decommissioning satellite:', error.message));
  };

  return (
    <div className="App">
      <h1>Space Maintenance</h1>
      {satellites.length > 0 ? (
        <ul>
          {satellites.map(satellite => (
            <li key={satellite._id}>
              {satellite.name}
              {" "}
              <button onClick={() => handleDecommission(satellite.name)}>Decommission</button>
            </li>
          ))}
        </ul>
      ) : (
        <p>No satellites found</p>
      )}
    </div>
  );
}

export default App;

Good day, Space Voyager!

We need to ensure our database contains only planets bigger than Earth. We want to delete all the planets with mass greater than 1, but something seems to be wrong with our current code, as it is deleting the wrong planets. Can you identify and fix this issue to maintain our cosmic database accurately?

In [ ]:
// index.js
const express = require('express');
const cors = require('cors');
const mongoose = require('mongoose');
const initialDBSetup = require('./initialDBSetup');

// Create a new express app
const app = express();

app.use(express.json());
app.use(cors({origin: 'http://localhost:3000'}));

// Connect to MongoDB
mongoose
  .connect('mongodb://127.0.0.1/myDB', { useNewUrlParser: true, useUnifiedTopology: true })
  .then(async () => {
    console.log('connected');
    // initialize DB
    await initialDBSetup();
  });

const planetSchema = new mongoose.Schema({
  name: String,
  size: String,
  mass: Number,
  distanceFromSun: String
});

const Planet = mongoose.model('Planet', planetSchema, 'planets');

app.post('/api/delete-planets', async (req, res) => {
  try {
    const result = await Planet.deleteMany({ mass: { $gt: 1 } });
    res.status(200).json({ message: `Deleted ${result.deletedCount} planets.` });
  } catch (error) {
    res.status(500).json({ message: 'Error deleting planets', error: error });
  }
});

app.listen(5000, () => console.log('Server running on port 5000'));

In [ ]:
import { useEffect, useState } from 'react';
import axios from 'axios';

const App = () => {
  const [message, setMessage] = useState();
  const [error, setError] = useState('');
  
  useEffect(() => {
    axios.post('/api/delete-planets')
      .then(response => setMessage(response.data.message))
      .catch(error => setError(error.response.message));
  }, []);

  if (error) {
    return <p>Error: {error}</p>
  } else {
    return (
      <div>
        <h1>Space Maintenance</h1>
        {message}
      </div>
    );
  }
}

export default App;

Let's keep our space data clean. You've already learned how to remove small planets. Now it's your turn to code the back-end server request that handles this task. Write the Express.js route that responds to a DELETE request aimed at clearing out small planets from our database.

In [ ]:
// Index.js (Express server)
const express = require('express');
const cors = require('cors');
const mongoose = require('mongoose');
const initialDBSetup = require('./initialDBSetup');

// Create a new express app
const app = express();

app.use(express.json());
app.use(cors({origin: 'http://localhost:3000'}));

// Connect to MongoDB
mongoose
  .connect('mongodb://127.0.0.1/myDB', { useNewUrlParser: true, useUnifiedTopology: true })
  .then(async () => {
    console.log('connected');
    // initialize DB
    await initialDBSetup();
  });

const Planet = mongoose.model('Planet', new mongoose.Schema({ name: String, size: String }));

app.use(express.json());

// TODO: Implement the DELETE route named '/api/decommission-planet' to decommission planets that have 'Small' size
app.delete('/api/decommission-planet/', async (req, res) => {
  try {
    const result = await Planet.deleteMany({ size: 'Small' }); 
    // TODO: Delete the small planets from the database and respond with the number decommissioned

    if (result.deletedCount > 0) {
      res.status(200).json({ message: `Deleted ${result.deletedCount} planets.` });
    } else {
      res.status(404).json({ message: 'Planet(s) not found.' });
    }
  } catch (error) {
    res.status(500).json({ message: 'Error deleting planets', error: error });
  }
});


// TODO: Catch possible errors and return error message with 500 status code

const port = 5000;
app.listen(port, () => console.log(`Server running on port ${port}`));

In [ ]:
import { useState } from 'react';
import axios from 'axios';

function App() {
  const [deletionMessage, setDeletionMessage] = useState('');

  const decommissionPlanet = () => {
    axios.delete('/api/decommission-planet')
      .then(response => setDeletionMessage(response.data))
      .catch(error => console.error('Error:', error));
  };

  return (
    <div>
      <button onClick={decommissionPlanet}>Decommission Small Planets</button>
      <p>{deletionMessage}</p>
    </div>
  );
}

export default App;

Stellar Navigator, your final mission is to write an endpoint that handles the deletion of a celestial body from our database. You have seen how it's done; now, venture forth and write the code from scratch.

Remember, you're setting sail from port 5000 and targeting the planets in our cosmic database for deletion. Good luck, and may your code be as clear as the night sky!

In [ ]:
// index.js (Express server)
const express = require('express');
const cors = require('cors');
const mongoose = require('mongoose');
const initialDBSetup = require('./initialDBSetup');

// Create a new express app
const app = express();

app.use(express.json());
app.use(cors({origin: 'http://localhost:3000'}));

// Connect to MongoDB
mongoose
  .connect('mongodb://127.0.0.1/myDB', { useNewUrlParser: true, useUnifiedTopology: true })
  .then(async () => {
    console.log('connected');
    // initialize DB
    await initialDBSetup();
  });

// TODO: Create the schema and model for planets with the following fields name(String), size(String), mass(String) and distanceFromSun(String)
const planetSchema = new mongoose.Schema({
  name: String,
  size: String,
  mass: Number,
  distanceFromSun: String
});

const Planet = mongoose.model('Planet', planetSchema, 'planets');

// TODO: Define a DELETE endpoint '/api/planets/delete/:name' to remove a planet by its name and return an object with success message
app.delete('/api/planets/delete/:name', async (req, res) => {
  try {
    const result = await Planet.deleteMany({ name : req.params.name }); // updated from isActive: false to name : req.params.name
    if (result.deletedCount > 0) {
      res.status(200).json({ message: `Successfully deleted Planet: ${req.params.name}` });
    } else {
      res.status(404).json({ message: 'Planet not found.' });
    }
  } catch (error) {
    res.status(500).json({ message: 'Failed to delete Planet.' });
  }
});

// TODO: Have the app listen on the defined port '5000' and log a message confirming the server is running
const port = 5000;
app.listen(port, () => console.log(`Server running on port ${port}`));

In [ ]:
import { useState, useEffect } from 'react';
import axios from 'axios';

function App() {
  const [deletedPlanet, setDeletedPlanet] = useState('');

  useEffect(() => {
    axios.delete('/api/planets/delete/Neptune')
      .then(response => setDeletedPlanet(response.data.message))
      .catch(error => console.log(error));
  }, []);

  return (
    <div>
      <p>{deletedPlanet}</p>
    </div>
  );
}

export default App;